In [64]:
#Table list , comma separated list ex. 'Table1,Table2' !!!CASE SENSITIVE!!!
tables = 'ContactPerson,CustClassificationGroup,CustGroup,CustInvoiceJour,CustInvoiceTrans,CustTable,DataArea,DimensionAttribute,\
DimensionAttributeDirCategory,DimensionAttributeValue,DimensionAttributeValueSetItem,DimensionFinancialTag,DirOrganizationName,\
DirPartyLocation,DirPartyRelationship,DirPartyTable,DirPersonName,EcoResProduct,EcoResProductTranslation,ForecastModel,ForecastSales,\
HcmWorker,InventBatch,InventDim,InventDimCombination,InventLocation,InventSite,InventSum,InventTable,InventTableModule,InventTrans,\
InventTransOrigin,Ledger,LineOfBusiness,LogisticsPostalAddress,MarkupTrans,OMOperatingUnit,SalesLine,SalesPool,SalesTable,smmBusRelChainGroup,\
smmBusRelSalesDistrictGroup,smmBusRelSegmentGroup,smmBusRelSubSegmentGroup,WHSInventStatus,WMSLocation,CustPackingSlipTrans,CustPackingSlipJour,\
Currency,ExchangeRate,ExchangeRateType,ExchangeRateCurrencyPair,SRSAnalysisEnums,BOM,BOMCalcTable,BOMCalcTrans,BOMTable,BOMVersion,\
EcoResCategory,EcoResProductMasterDimensionValue,PurchLine,PurchTable,VendGroup,VendPackingSlipJour,VendPackingSlipTrans,VendTable'


#tables = 'BOMVersion,EcoResCategory'

#Storage account details where data feed exports are stored
setupFilesContainer = "abfss://dynamics365-financeandoperations@mcasandbox01d365foadl.dfs.core.windows.net"
setupFilesFolderPath = "/mca-sandbox-01.sandbox.operations.dynamics.com/"
containerName = "dynamics365-financeandoperations"
storageAccount = "mcasandbox01d365foadl"

#linked service name to storage where data feed exports are stored
linkedServiceName = "FnO_DataLakeStorage"

#Checkpointing configurations
CheckpointcontainerName = "synapse"
CheckpointstorageAccount = "mcainspiredev02syn"
ChangeFeedCheckPointRoot = "ChangeFeedStructuredStreamingCheckpoint"
FullCheckPointRoot = "StructuredStreamingCheckpoint"

#Delta Databases
bronzedatabaseName = "d365_bronze"
silverdatabaseName = "d365_silver"
golddatabaseName = "combined_gold"
configsdatabaseName = "configs"

#Synapse Storage account details
targetStorageAccountName = "mcainspiredev02syn"
targetStorageContainerName = "synapse"
rootTargetTablePath = "synapse/workspaces/mca-inspiredev-02-syn/warehouse/d365_bronze.db/"

#Pipeline Grouping Name - Core reserved for core ETL for Inspire Platform
pipelineGroup = "Core"

#Initial Setup Flag
isInitialSetup = 0

In [65]:
#Create configuration tables if not exists

configsSchema = "CREATE DATABASE IF NOT EXISTS %s" % (configsdatabaseName)
spark.sql(configsSchema)

bronzeSchema = "CREATE DATABASE IF NOT EXISTS %s" % (bronzedatabaseName)
spark.sql(bronzeSchema)

silverSchema = "CREATE DATABASE IF NOT EXISTS %s" % (silverdatabaseName)
spark.sql(silverSchema)

goldSchema = "CREATE DATABASE IF NOT EXISTS %s" % (golddatabaseName)
spark.sql(goldSchema)


bronzemetatable = "CREATE TABLE IF NOT EXISTS %s.bronzemetatable ( \
SourceTableName string, \
SourceStorageAccountName string, \
SourceStorageContainerName string, \
SourceChangeFeedManifestPath string, \
SourceFullTablesManifestPath string, \
SourceChangeFeedFilePath string, \
SourceFullTablesFilePath string, \
CheckpointAccountName string, \
CheckpointContainerName string, \
ChangeFeedCheckpointPath string, \
FullCheckpointPath string, \
TargetDatabaseName string, \
TargetTableName string, \
TargetStorageAccountName string, \
TargetStorageContainerName string, \
TargetTablePath string, \
IsActive int, \
IsProcessingComplete int, \
PipelineGroup string, \
ETLBatchSeriesID bigint) \
USING DELTA \
PARTITIONED by (PipelineGroup)" % (configsdatabaseName)

etlbatchseries = "CREATE TABLE IF NOT EXISTS %s.etlbatchseries (ETLBatchSeriesID bigint, \
ETLBatchSeriesCreateDate timestamp, \
IsProcessingComplete tinyint, \
PipelineGroup string, \
IsFullLoad int) \
USING DELTA \
PARTITIONED by (PipelineGroup)" % (configsdatabaseName)

silvermeta = "CREATE TABLE IF NOT EXISTS %s.silvermeta (Database string, \
TableName string, \
SourceQuery string, \
MergeJoinKey string, \
IsActive int, \
IsProcessingComplete int, \
PipelineGroup string, \
ETLBatchSeriesID bigint) \
USING DELTA \
PARTITIONED by (PipelineGroup)" % (configsdatabaseName)

goldmeta = "CREATE TABLE IF NOT EXISTS %s.goldmeta (Database string, \
TableName string, \
SourceQuery string, \
MergeJoinKey string, \
PartitionKey string, \
IsProcessingComplete int, \
SeriesOrdinal int, \
IsIncrementalOnly int, \
PipelineGroup string, \
ETLBatchSeriesID bigint) \
USING DELTA \
PARTITIONED BY (PipelineGroup)" % (configsdatabaseName)


bronzeconstantentity = "CREATE TABLE IF NOT EXISTS %s.constantentity (TableName string, \
FieldName string, \
Constant0 string, \
FormDisplayName string, \
Value string, \
Key string, \
ETLBatchSeriesID bigint, \
DML_Action string) \
USING DELTA " % (bronzedatabaseName)

silverconstantentity = "CREATE TABLE IF NOT EXISTS %s.constantentity ( \
TableName string, \
FieldName string, \
Constant0 string, \
FormDisplayName string, \
Value string, \
Key string, \
ETLBatchSeriesID bigint, \
DML_Action string, \
ETLIsDeleted int) \
USING DELTA " % (silverdatabaseName)

spark.sql(bronzemetatable)
spark.sql(etlbatchseries)
spark.sql(silvermeta)
spark.sql(goldmeta)
spark.sql(bronzeconstantentity)
spark.sql(silverconstantentity)

In [66]:
def Convert(string):
    li = list(string.split(","))
    return li
tableList = Convert(tables)
coreTableList = tableList
for tb in tableList:
    try:
        spark.sql("DESCRIBE %s.%s" % (bronzedatabaseName,tb))
        spark.sql("DESCRIBE %s.%s" % (silverdatabaseName,tb))
        tableList.remove(tb)
            
    except:
        a=""
if pipelineGroup == "Core":
    shiftBronzeConfigToCore = "UPDATE %s.bronzemetatable \
    SET PipelineGroup = 'Core', \
        IsActive = 1 \
    WHERE lower(TargetTableName) IN (%s)" % (configsdatabaseName,str(coreTableList).replace('[','').replace(']','').lower())

    shiftSilverConfigToCore = "UPDATE %s.silvermeta \
    SET PipelineGroup = 'Core', \
        IsActive = 1 \
    WHERE lower(TableName) IN (%s)" % (configsdatabaseName,str(coreTableList).replace('[','').replace(']','').lower())
    spark.sql(shiftBronzeConfigToCore)
    spark.sql(shiftSilverConfigToCore)
if len(tableList) == 0:
    mssparkutils.notebook.exit("Tables already exist in bronze and silver. Skipping import and exiting notebook execution. No work performed.")

In [67]:
def left(s, amount):
    return s[:amount]

def right(s, amount):
    return s[-amount:]

abfssString = "abfss://" + containerName + "@" + storageAccount + ".dfs.core.windows.net"

In [68]:
import os
import glob
from pyspark.sql import DataFrame
from pyspark.sql.types import StringType
final_df = spark.sql("SELECT 'EMPTY' as Empty")

mssparkutils.fs.unmount("/Tables_mnt")

mssparkutils.fs.mount(
    setupFilesContainer,
    "/Tables_mnt",
    {"linkedService" : linkedServiceName}
)

jbID = mssparkutils.env.getJobId()
path='/synfs/' + jbID + '/Tables_mnt' + setupFilesFolderPath
replacePath='/synfs/' + jbID + '/Tables_mnt'
result = []
fullResult = []
for x in os.walk(path):
    for y in glob.glob(os.path.join(x[0], '*manifest.cdm.json')):
        if "ChangeFeed" not in y and "/Custom/" not in y:
            fullResult.append(y)          
        if "ChangeFeed" in y:
            result.append(y)     



In [ ]:
from pyspark.sql.types import StructType,StructField, StringType, IntegerType
df = spark.createDataFrame(result, StringType())
dataCollect = df.collect()

schema = StructType([ \
    StructField("Account",StringType(),True), \
    StructField("Manifest",StringType(),True), \
    StructField("Entity",StringType(),True)])

dfExists = 0


for row in dataCollect:   
    configPath = row['value'].replace(replacePath,setupFilesContainer)
    
    path = configPath[::-1]
    startString = int(path.find("/")+ 1)
    stepString = 10
    path = path[path.find("/")+ 1:len(path)]
    path = path[::-1]

    df1 = spark.read.json(configPath, multiLine=True)
    i=0
    try:
        for subManifest in df1.select(df1['entities']).take(1):
            while i<len(subManifest[0]):
                if i > 1000:
                    break
                entityName =subManifest[0][i]['entityName']
                if  len(tableList) > 0:
                    if entityName in tableList:
                        data = [(setupFilesContainer.replace("abfss://","").replace('dynamics365-financeandoperations@',''),"dynamics365-financeandoperations"+configPath.replace(setupFilesContainer,''), entityName)]
                        if dfExists == 0:
                            dfManifest = spark.createDataFrame(data=data,schema=schema)
                            dfExists = 1
                        else:
                            dfManifestAppend = spark.createDataFrame(data=data,schema=schema)
                            dfManifest = dfManifest.union(dfManifestAppend)
                else:
                    data = [(setupFilesContainer.replace("abfss://","").replace('dynamics365-financeandoperations@',''),"dynamics365-financeandoperations"+configPath.replace(setupFilesContainer,''), entityName)]
                    if dfExists == 0:
                        dfManifest = spark.createDataFrame(data=data,schema=schema)
                        dfExists = 1
                    else:
                        dfManifestAppend = spark.createDataFrame(data=data,schema=schema)
                        dfManifest = dfManifest.union(dfManifestAppend)
                i=i+1
            
    except :
        print("Following filepath does not contain submanifest values. Skipping. " + configPath)
    if i>1000:
        raise Exception("Error: Submanifest list exceeded 1000 entityName values.") 

In [ ]:
from pyspark.sql.types import StructType,StructField, StringType, IntegerType

dfExists = 0

def left(s, amount):
    return s[:amount]

def right(s, amount):
    return s[-amount:]

schema = StructType([ \
    StructField("SourceTableName",StringType(),True), \
    StructField("SourceStorageAccountName",StringType(),True), \
    StructField("SourceStorageContainerName",StringType(),True), \
    StructField("SourceChangeFeedManifestPath",StringType(),True), \
    StructField("SourceFullTablesManifestPath",StringType(),True), \
    StructField("SourceChangeFeedFilePath",StringType(),True), \
    StructField("SourceFullTablesFilePath",StringType(),True), \
    StructField("CheckpointAccountName",StringType(),True), \
    StructField("CheckpointContainerName",StringType(),True), \
    StructField("ChangeFeedCheckpointPath",StringType(),True), \
    StructField("FullCheckpointPath",StringType(),True),
    StructField("TargetDatabaseName",StringType(),True),
    StructField("TargetTableName",StringType(),True),
    StructField("TargetStorageAccountName",StringType(),True),
    StructField("TargetStorageContainerName",StringType(),True),
    StructField("TargetTablePath",StringType(),True), \
    StructField("IsActive",IntegerType(),True), \
    StructField("IsProcessingComplete",IntegerType(),True)])
dataChangeFeed=[]

dfManifestCollect = dfManifest.collect()

for colManifest in dfManifestCollect:
    #storage = colManifest["Account"]
    manifestPath = colManifest["Manifest"]
    entity = colManifest["Entity"]
    
    manifestPath = abfssString + right(manifestPath,(len(manifestPath)-len(containerName)))
    
    revmanifestPath = manifestPath[len(manifestPath)::-1]
    ordVal = revmanifestPath.find("/")
    substringrevmanifestPath = right(revmanifestPath,(len(revmanifestPath) -ordVal - 1))
    substringmanifestPath = substringrevmanifestPath[len(substringrevmanifestPath)::-1]
    
    sourceChangeFeedManifestPath =  right(manifestPath,(len(manifestPath)-(len(abfssString)+1)))
    df1 = spark.read.json(manifestPath, multiLine=True)
    entityPaths = df1.select("entities").limit(1).collect()[0][0]
    for entity in entityPaths:
        if  len(tableList) > 0:
            if entity["entityName"] in tableList:
                entityName = entity["entityName"]
                #sourceChangeFeedFilePath = substringmanifestPath + "/" + entityName
                sourceChangeFeedFilePath = (substringmanifestPath + "/" + entityName).replace(abfssString,"")
                targetTablePath = rootTargetTablePath + entityName.lower()
                entityPath = entity["entityPath"]
                targetTableName = entityName
                entityManifestPath = left(entityPath,(len(entityPath) - (len(entityName)+1)))
                entityFilePath = substringmanifestPath + "/"+ entityName
                dataChangeFeed.append((entityName,storageAccount,containerName,sourceChangeFeedManifestPath,"",right(sourceChangeFeedFilePath,len(sourceChangeFeedFilePath)-1),"",CheckpointstorageAccount,CheckpointcontainerName,ChangeFeedCheckPointRoot+"/"+entityName,FullCheckPointRoot+"/"+entityName,bronzedatabaseName,targetTableName,targetStorageAccountName,targetStorageContainerName,targetTablePath,1,1))
        else:
            entityName = entity["entityName"]
            #sourceChangeFeedFilePath = substringmanifestPath + "/" + entityName
            sourceChangeFeedFilePath = (substringmanifestPath + "/" + entityName).replace(abfssString,"")
            targetTablePath = rootTargetTablePath + entityName.lower()
            entityPath = entity["entityPath"]
            targetTableName = entityName
            entityManifestPath = left(entityPath,(len(entityPath) - (len(entityName)+1)))
            entityFilePath = substringmanifestPath + "/"+ entityName
            dataChangeFeed.append((entityName,storageAccount,containerName,sourceChangeFeedManifestPath,"",right(sourceChangeFeedFilePath,len(sourceChangeFeedFilePath)-1),"",CheckpointstorageAccount,CheckpointcontainerName,ChangeFeedCheckPointRoot+"/"+entityName,FullCheckPointRoot+"/"+entityName,bronzedatabaseName,targetTableName,targetStorageAccountName,targetStorageContainerName,targetTablePath,1,1))

In [ ]:
spark.sql("DROP TABLE IF EXISTS %s.bronzeMeta2" % (configsdatabaseName))
rdd = sc.parallelize(dataChangeFeed)
dfBronzeConfigurations = sqlContext.createDataFrame(rdd, schema)  
dfBronzeConfigurations.createOrReplaceTempView("vwBronzeChangeFeed")
dfBronzeConfigurations = spark.sql("SELECT DISTINCT * FROM vwBronzeChangeFeed")
configsTableName = configsdatabaseName + ".bronzeMeta2"
dfBronzeConfigurations.write.mode("overwrite").format("delta").saveAsTable(configsTableName)

In [ ]:
from pyspark.sql.types import StructType,StructField, StringType, IntegerType
df = spark.createDataFrame(fullResult, StringType())
dataCollect = df.collect()

schema = StructType([ \
    StructField("Account",StringType(),True), \
    StructField("Manifest",StringType(),True), \
    StructField("Entity",StringType(),True)])

dfExists = 0


for row in dataCollect:   
    configPath = row['value'].replace(replacePath,setupFilesContainer)
    
    path = configPath[::-1]
    startString = int(path.find("/")+ 1)
    stepString = 10
    path = path[path.find("/")+ 1:len(path)]
    path = path[::-1]
    i=0
    df1 = spark.read.json(configPath, multiLine=True)    
    try:
        for subManifest in df1.select(df1['entities']).take(1):
            while i<len(subManifest[0]):
                if i > 1000:
                    break
                entityName =subManifest[0][i]['entityName']
                if  len(tableList) > 0:
                    if entityName in tableList:
                        data = [(setupFilesContainer.replace("abfss://","").replace('dynamics365-financeandoperations@',''),"dynamics365-financeandoperations"+configPath.replace(setupFilesContainer,''), entityName)]
                        if dfExists == 0:
                            dfManifest = spark.createDataFrame(data=data,schema=schema)
                            dfExists = 1
                        else:
                            dfManifestAppend = spark.createDataFrame(data=data,schema=schema)
                            dfManifest = dfManifest.union(dfManifestAppend)
                else:
                    data = [(setupFilesContainer.replace("abfss://","").replace('dynamics365-financeandoperations@',''),"dynamics365-financeandoperations"+configPath.replace(setupFilesContainer,''), entityName)]
                    if dfExists == 0:
                        dfManifest = spark.createDataFrame(data=data,schema=schema)
                        dfExists = 1
                    else:
                        dfManifestAppend = spark.createDataFrame(data=data,schema=schema)
                        dfManifest = dfManifest.union(dfManifestAppend)
                i=i+1
            
    except :
        print("Following filepath does not contain submanifest values. Skipping. " + configPath)
    if i>1000:
        raise Exception("Error: Submanifest list exceeded 1000 entityName values.")       


In [ ]:
from pyspark.sql.types import StructType,StructField, StringType, IntegerType

dfExists = 0

def left(s, amount):
    return s[:amount]

def right(s, amount):
    return s[-amount:]

schema = StructType([ \
    StructField("SourceTableName",StringType(),True), \
    StructField("SourceStorageAccountName",StringType(),True), \
    StructField("SourceStorageContainerName",StringType(),True), \
    StructField("SourceChangeFeedManifestPath",StringType(),True), \
    StructField("SourceFullTablesManifestPath",StringType(),True), \
    StructField("SourceChangeFeedFilePath",StringType(),True), \
    StructField("SourceFullTablesFilePath",StringType(),True), \
    StructField("CheckpointAccountName",StringType(),True), \
    StructField("CheckpointContainerName",StringType(),True), \
    StructField("ChangeFeedCheckpointPath",StringType(),True), \
    StructField("FullCheckpointPath",StringType(),True), \
    StructField("TargetDatabaseName",StringType(),True), \
    StructField("TargetTableName",StringType(),True), \
    StructField("TargetStorageAccountName",StringType(),True), \
    StructField("TargetStorageContainerName",StringType(),True), \
    StructField("TargetTablePath",StringType(),True), \
    StructField("IsActive",IntegerType(),True), \
    StructField("IsProcessingComplete",IntegerType(),True)])


dfManifestCollect = dfManifest.collect()
data=[]
for colManifest in dfManifestCollect:
    storage = colManifest["Account"]
    manifestPath = colManifest["Manifest"]
    entity = colManifest["Entity"]
    
    manifestPath = abfssString + right(manifestPath,(len(manifestPath)-len(containerName)))
    #filePath = right(manifestPath,(len(manifestPath)-len(containerName)))
    revmanifestPath = manifestPath[len(manifestPath)::-1]
    ordVal = revmanifestPath.find("/")
    substringrevmanifestPath = right(revmanifestPath,(len(revmanifestPath) -ordVal - 1))
    substringmanifestPath = substringrevmanifestPath[len(substringrevmanifestPath)::-1]

    sourceFullTableManifestPath =  right(manifestPath,(len(manifestPath)-(len(abfssString)+1)))
    df1 = spark.read.json(manifestPath, multiLine=True)
    entityPaths = df1.select("entities").limit(1).collect()[0][0]
    for entity in entityPaths:
        if len(tableList) > 0:
            if entity["entityName"] in tableList:
                entityName = entity["entityName"]
                sourceFullTableFilePath = (substringmanifestPath + "/" + entityName).replace(abfssString,"")
                targetTablePath = rootTargetTablePath + entityName.lower()
                entityPath = entity["entityPath"]
                targetTableName = entityName
                entityManifestPath = left(entityPath,(len(entityPath) - (len(entityName)+1)))
                #entityConfigurationPath = substringmanifestPath + "/"+ entityManifestPath
                entityFilePath = substringmanifestPath + "/"+ entityName
                
                data.append((entityName,storageAccount,containerName,"",sourceFullTableManifestPath,"",right(sourceFullTableFilePath,len(sourceFullTableFilePath)-1),CheckpointstorageAccount,CheckpointcontainerName,ChangeFeedCheckPointRoot+"/"+entityName,FullCheckPointRoot+"/"+entityName,bronzedatabaseName,targetTableName,targetStorageAccountName,targetStorageContainerName,targetTablePath,1,1))
        else:
            entityName = entity["entityName"]
            sourceFullTableFilePath = (substringmanifestPath + "/" + entityName).replace(abfssString,"")
            targetTablePath = rootTargetTablePath + entityName.lower()
            entityPath = entity["entityPath"]
            targetTableName = entityName
            entityManifestPath = left(entityPath,(len(entityPath) - (len(entityName)+1)))
            #entityConfigurationPath = substringmanifestPath + "/"+ entityManifestPath
            entityFilePath = substringmanifestPath + "/"+ entityName
            
            data.append((entityName,storageAccount,containerName,"",sourceFullTableManifestPath,"",right(sourceFullTableFilePath,len(sourceFullTableFilePath)-1),CheckpointstorageAccount,CheckpointcontainerName,ChangeFeedCheckPointRoot+"/"+entityName,FullCheckPointRoot+"/"+entityName,bronzedatabaseName,targetTableName,targetStorageAccountName,targetStorageContainerName,targetTablePath,1,1))

In [ ]:
spark.sql("DROP TABLE IF EXISTS %s.bronzeMeta3" % (configsdatabaseName))
rdd = sc.parallelize(data)
dfFullBronzeConfigurations = sqlContext.createDataFrame(rdd, schema)  
dfFullBronzeConfigurations.createOrReplaceTempView("vwBronzeFull")
dfFullBronzeConfigurations = spark.sql("SELECT DISTINCT * FROM vwBronzeFull")
configsTableName = configsdatabaseName + ".bronzeMeta3"
dfFullBronzeConfigurations.write.mode("overwrite").format("delta").saveAsTable(configsTableName)

In [ ]:
sqlCommand = "SELECT A.SourceTableName,A.SourceStorageAccountName,A.SourceStorageContainerName,B.SourceChangeFeedManifestPath,A.SourceFullTablesManifestPath,B.SourceChangeFeedFilePath,A.SourceFullTablesFilePath,A.CheckpointAccountName,A.CheckpointContainerName,A.ChangeFeedCheckpointPath,A.FullCheckpointPath,A.TargetDatabaseName,A.TargetTableName,A.TargetStorageAccountName,A.TargetStorageContainerName,A.TargetTablePath, 1 AS IsActive, A.IsProcessingComplete,'%s' AS PipelineGroup, CAST(0 AS BIGINT) AS ETLBatchSeriesID FROM  %s.bronzeMeta3 A INNER JOIN %s.bronzeMeta2 B ON A.SourceTableName = B.SourceTableName" % (pipelineGroup,configsdatabaseName,configsdatabaseName)
dfConfigsBronzeMetaTable = spark.sql(sqlCommand)
dfConfigsBronzeMetaTable.createOrReplaceTempView("vwBronzeMetaTable")
configsTableName = configsdatabaseName + ".BronzeMetaTable"

sqlMerge = "MERGE INTO %s T \
USING vwBronzeMetaTable S ON T.TargetTableName = S.TargetTableName AND T.PipelineGroup = S.PipelineGroup \
WHEN MATCHED THEN UPDATE SET * \
WHEN NOT MATCHED THEN INSERT *" % (configsTableName)
spark.sql(sqlMerge)

In [ ]:
if len(tableList) > 0:
    sqlCommand = "SELECT DISTINCT * FROM %s WHERE IsActive = 1 AND PipelineGroup = '%s' AND lower(TargetTableName) IN (%s) ORDER BY SourceTableName" % (configsTableName,pipelineGroup,str(tableList).replace('[','').replace(']','').lower())
else:
    sqlCommand = "SELECT DISTINCT * FROM %s WHERE IsActive = 1 AND PipelineGroup = '%s' ORDER BY SourceTableName" % (configsTableName,pipelineGroup)


dfSelect = spark.sql(sqlCommand)
dfSelectCollect = dfSelect.collect()
for table in dfSelectCollect:
    #print(table["SourceChangeFeedManifestPath"])
    tableName = table["SourceTableName"]
    sourceManifestPath = table["SourceChangeFeedManifestPath"]
    
    abfssString = "abfss://" + table["SourceStorageContainerName"] + "@" + table["SourceStorageAccountName"]  + ".dfs.core.windows.net"
    sourceManifestPath = abfssString + "/" + sourceManifestPath
    #print(sourceManifestPath)
    
    revsourceManifestPath = sourceManifestPath[len(sourceManifestPath)::-1]
    ordVal = revsourceManifestPath.find("/")
    substringrevsourceManifestPath = right(revsourceManifestPath,(len(revsourceManifestPath) -ordVal - 1))
    substringsourceManifestPath = substringrevsourceManifestPath[len(substringrevsourceManifestPath)::-1]

    #print(substringsourceManifestPath)
    df1 = spark.read.json(sourceManifestPath, multiLine=True)
    entityPaths = df1.select("entities").limit(1).collect()[0][0]
    
    for entity in entityPaths:
        
        if entity["entityName"]==tableName:
            
            entityName = entity["entityName"]
            entityPath = entity["entityPath"]
            entityManifestPath = left(entityPath,(len(entityPath) - (len(entityName)+1)))
            entityConfigurationPath = substringsourceManifestPath + "/"+ entityManifestPath
            
            jsonConfigs = spark.read.json(entityConfigurationPath, multiLine=True)
            definitions = jsonConfigs.select("definitions").limit(1).collect()[0][0]
            sqlCreate = "CREATE OR REPLACE TABLE " + bronzedatabaseName + "." + tableName + " ("
            colList = ''
            rowNum = 0
            schemaList = []
            for rows in definitions:
                fields = rows["hasAttributes"]
                for field in fields:
                    fieldName = field["name"]
                    dataFormat = field["dataFormat"].replace("Binary","String").replace("DateTime","Timestamp").replace("Int64","BIGINT").replace("Int32","INT").replace("Decimal","Decimal(32,4)").replace("Guid","String")
                    if rowNum == 0:
                        colList = fieldName + " " + dataFormat
                        rowNum = 1
                    else:
                        colList = colList + ", " + fieldName + " " + dataFormat
                    schemaList.append((fieldName,dataFormat))

                sqlCreate = sqlCreate + colList + ", ETLBatchSeriesID BIGINT) USING DELTA"
                sqlSilverCreate = "CREATE OR REPLACE TABLE " + silverdatabaseName + "." + tableName + " (" + colList + ", ETLBatchSeriesID BIGINT, ETLIsDeleted INT) USING DELTA"
                #print(tableName)
                spark.sql(sqlCreate) 
                spark.sql(sqlSilverCreate)
                #print(sqlCreate)
                #print(sqlSilverCreate)
if pipelineGroup == 'Core':
    mergeSQL = "MERGE INTO %s.silvermeta T \
    USING ( \
    SELECT '%s' as Database, TargetTableName as TableName,concat('SELECT * FROM (SELECT *, RANK() OVER(PARTITION BY RECID ORDER BY CAST(Start_LSN as BINARY) DESC,CAST(Seq_Val as BINARY) DESC, ETLBatchSeriesID DESC) as TopRank FROM ', TargetDatabaseName, '.', TargetTableName, ' WHERE ETLBatchSeriesID >= @REPLACEBATCH@ ) a WHERE TopRank = 1') as SourceQuery,'s.`RECID` = t.`RECID`' as MergeJoinKey, IsActive, 1 as IsProcessingComplete, '%s' as PipelineGroup, CAST(0 AS BIGINT) AS ETLBatchSeriesID \
    FROM %s.bronzemetatable WHERE IsActive = 1 AND PipelineGroup = '%s' AND lower(TargetTableName) IN (%s) \
    UNION \
    SELECT '%s' as Database, 'ConstantEntity', 'SELECT TableName, FieldName, Constant0, FormDisplayName, MAX(Value) AS Value, Key, ETLBatchSeriesID, DML_Action FROM (SELECT *, RANK() OVER(PARTITION BY TableName,FieldName,Constant0,Key ORDER BY ETLBatchSeriesID DESC) as TopRank FROM %s.ConstantEntity WHERE ETLBatchSeriesID >= @REPLACEBATCH@ ) a WHERE TopRank = 1 GROUP BY TableName, FieldName, Constant0, FormDisplayName, Key, ETLBatchSeriesID, DML_Action','s.`TableName` = t.`TableName` AND s.`FieldName`= t.`FieldName` AND s.`Constant0` = t.`Constant0` and s.`Key` = t.`Key`',1, 1 as IsProcessingComplete, '%s' as PipelineGroup, CAST(0 AS BIGINT) AS ETLBatchSeriesID) AS S \
    ON T.TableName = S.TableName AND T.PipelineGroup = S.PipelineGroup \
    WHEN NOT MATCHED THEN INSERT *" % (configsdatabaseName,silverdatabaseName,pipelineGroup,configsdatabaseName,pipelineGroup,str(tableList).replace('[','').replace(']','').lower(),silverdatabaseName,bronzedatabaseName,pipelineGroup)
else:
    mergeSQL = "MERGE INTO %s.silvermeta T \
    USING ( \
    SELECT '%s' as Database, TargetTableName as TableName,concat('SELECT * FROM (SELECT *, RANK() OVER(PARTITION BY RECID ORDER BY CAST(Start_LSN as BINARY) DESC,CAST(Seq_Val as BINARY) DESC, ETLBatchSeriesID DESC) as TopRank FROM ', TargetDatabaseName, '.', TargetTableName, ' WHERE ETLBatchSeriesID >= @REPLACEBATCH@ ) a WHERE TopRank = 1') as SourceQuery,'s.`RECID` = t.`RECID`' as MergeJoinKey, IsActive, 1 as IsProcessingComplete, '%s' as PipelineGroup, CAST(0 AS BIGINT) AS ETLBatchSeriesID \
    FROM %s.bronzemetatable WHERE IsActive = 1 AND PipelineGroup = '%s' AND lower(TargetTableName) IN (%s)) S \
    ON T.TableName = S.TableName AND T.PipelineGroup = S.PipelineGroup \
    WHEN NOT MATCHED THEN INSERT *" % (configsdatabaseName,silverdatabaseName,pipelineGroup,configsdatabaseName,pipelineGroup,str(tableList).replace('[','').replace(']','').lower())
spark.sql(mergeSQL)

In [ ]:
#####################################
#####   CONFIGURATION VALUES    #####
#####################################
if isInitialSetup != 1:
    retryAttempts = 1
    isFullLoad = 1
    parallelWorkers = 8

    df = spark.sql("SELECT MAX(ETLBatchSeriesID) + 1 AS ETLBatchSeriesID, COUNT(*) as RowCnt FROM %s.ETLBatchSeries WHERE PipelineGroup = '%s'" % (configsdatabaseName,pipelineGroup))

    RowCnt = df.select("RowCnt").limit(1).collect()[0][0]

    if RowCnt == 0:
        ETLBatchSeriesID = 1
        sqlQry= "INSERT INTO %s.ETLBatchSeries (ETLBatchSeriesID,ETLBatchSeriesCreateDate,IsProcessingComplete,PipelineGroup,IsFullLoad) SELECT 1, current_timestamp(),0,'%s',%i" % (configsdatabaseName,pipelineGroup,isFullLoad)
    else:
        ETLBatchSeriesID = df.select("ETLBatchSeriesID").limit(1).collect()[0][0]
        sqlQry= "INSERT INTO %s.ETLBatchSeries (ETLBatchSeriesID,ETLBatchSeriesCreateDate,IsProcessingComplete,PipelineGroup,IsFullLoad) SELECT '%i', current_timestamp(),0,'%s',%i" % (configsdatabaseName,ETLBatchSeriesID,pipelineGroup,isFullLoad)
    spark.sql(sqlQry)

    df = spark.sql("SELECT MAX(ETLBatchSeriesID) AS ETLBatchSeriesID FROM %s.ETLBatchSeries WHERE PipelineGroup = '%s'" % (configsdatabaseName,pipelineGroup))

    ETLBatchSeriesID = int(df.select("ETLBatchSeriesID").limit(1).collect()[0][0])
    sqlCommand = "SELECT * FROM %s.bronzeMetaTable WHERE IsActive = 1 and lower(TargetTableName) IN (%s) AND PipelineGroup = '%s'" % (configsdatabaseName,str(tableList).replace('[','').replace(']','').lower(),pipelineGroup)
    df=spark.sql(sqlCommand)
    bronzedfcollect = df.collect()   

from concurrent.futures import ThreadPoolExecutor
from datetime import datetime
from pyspark.sql.types import StructType,StructField, StringType, IntegerType
import json
resultList = []

def main():
    notebooks = list(gather_notebookcalls(bronzedfcollect,isFullLoad))
    with ThreadPoolExecutor(max_workers=parallelWorkers) as Thready:
        print("Thready")
        Thready.map(call_notebooks, notebooks)    

def call_notebooks(notebook_par: str) -> None:
    retry = retryAttempts
    notebookParams = json.loads(notebook_par)
    res=""
    try:
        spark.conf.set("spark.synapse.nbs.kernelid", "")
        res = mssparkutils.notebook.run(notebookParams["path"], notebookParams["timeout"], notebookParams["parameters"])
        resultList.append(res)
        print(res)
        IsRetry = 0
    except Exception as e:
        print(e)
        IsRetry = 1
    if IsRetry == 1 and retry > 0:
            print("Retrying")
            spark.conf.set("spark.synapse.nbs.kernelid", "")
            res = mssparkutils.notebook.run(notebookParams["path"], notebookParams["timeout"], notebookParams["parameters"])
            resultList.append(res)
            print(res) 
            retry = retry - 1
    else:
        raise ValueError("ERROR DURING PROCESSING. CHECK YOUR LOGS. " + notebookParams["parameters"])
    
  

  

def gather_notebookcalls(bronzedfcollect,isFullLoad) -> iter:  
    retryAttempts = 1
    notebooks = []
    schema=StructType([ \
        StructField("path",StringType(),True), \
        StructField("timeout",IntegerType(),True), \
        StructField("parameters",StructType([ \
            StructField("isFullLoad", IntegerType(), True),  \
            StructField("targetDatabaseName", StringType(), True), \
            StructField("targetTableName", StringType(), True), \
            StructField("targetTablePath", StringType(), True), \
            StructField("targetStorageContainerName", StringType(), True), \
            StructField("targetStorageAccountName", StringType(), True), \
            StructField("sourceStorageAccountName", StringType(), True), \
            StructField("sourceStorageContainerName", StringType(), True), \
            StructField("sourceManifestPath", StringType(), True), \
            StructField("sourceFilePath", StringType(), True), \
            StructField("checkpointAccountName", StringType(), True), \
            StructField("checkpointContainerName", StringType(), True), \
            StructField("checkpointPath", StringType(), True), \
            StructField("mergeKey", StringType(), True), \
            StructField("ETLBatchSeriesID", IntegerType(), True)])), \
        StructField("retry",StringType(),True)])
    for row in bronzedfcollect:
        if isFullLoad == 1:
            targetDatabaseName=row['TargetDatabaseName']
            targetTableName=row['TargetTableName']
            targetTablePath=row['TargetTablePath']
            sourceStorageContainerName=row['SourceStorageContainerName']
            sourceStorageAccountName=row['SourceStorageAccountName']
            sourceManifestPath=row['SourceFullTablesManifestPath']
            sourceFilePath=row['SourceFullTablesFilePath']
            checkpointAccountName=row['CheckpointAccountName']
            checkpointContainerName=row['CheckpointContainerName']
            checkpointPath=row['FullCheckpointPath']
            targetStorageContainerName=row['TargetStorageContainerName']
            targetStorageAccountName=row['TargetStorageAccountName']
            mergeKey="1=2"
        else:
            targetDatabaseName=row['TargetDatabaseName']
            targetTableName=row['TargetTableName']
            targetTablePath=row['TargetTablePath']
            sourceStorageContainerName=row['SourceStorageContainerName']
            sourceStorageAccountName=row['SourceStorageAccountName']
            sourceManifestPath=row['SourceChangeFeedManifestPath']
            sourceFilePath=row['SourceChangeFeedFilePath']
            checkpointAccountName=row['CheckpointAccountName']
            checkpointContainerName=row['CheckpointContainerName']
            checkpointPath=row['ChangeFeedCheckpointPath']
            targetStorageContainerName=row['TargetStorageContainerName']
            targetStorageAccountName=row['TargetStorageAccountName']
            mergeKey="s.recid=t.recid"
        notebooks.append(("/StgToBronze",5000,(isFullLoad,targetDatabaseName,targetTableName,targetTablePath,targetStorageContainerName,targetStorageAccountName,sourceStorageAccountName,sourceStorageContainerName,sourceManifestPath,sourceFilePath,checkpointAccountName,checkpointContainerName,checkpointPath,mergeKey,ETLBatchSeriesID),retryAttempts))
    rdd = sc.parallelize(notebooks)
    dfBronzeConfigurations = sqlContext.createDataFrame(rdd, schema)  
    notebookCollect = dfBronzeConfigurations.toJSON().collect()
    for notebook in notebookCollect:
        yield notebook

if __name__ == '__main__':
    t1 = datetime.now()
    if isInitialSetup != 1:
        main()
    t2 = datetime.now()
    x = t2-t1
    print(f'It took {x} to process files')

In [ ]:
updateBronzeMeta = "UPDATE %s.bronzemetatable \
SET ETLBatchSeriesID = %s \
WHERE IsActive = 1 and lower(TargetTableName) IN (%s) AND PipelineGroup = '%s'" % (configsdatabaseName,str(ETLBatchSeriesID),str(tableList).replace('[','').replace(']','').lower(),pipelineGroup)

spark.sql(updateBronzeMeta)
    

In [ ]:
import json
from pyspark.sql.types import StructType,StructField, StringType, IntegerType, LongType
if isInitialSetup != 1:
    if pipelineGroup == "Core":
        dfConfigurations = spark.sql("SELECT * FROM %s.bronzeMetaTable WHERE IsActive = 1 AND PipelineGroup = 'Core'" % (configsdatabaseName))
        dfConfigurationsCollect = dfConfigurations.collect()
        enumValues = []    
        DMLAction = "INSERT"
        for table in dfConfigurationsCollect:
            tableName = table["SourceTableName"]
            abfssString = "abfss://" + table["SourceStorageContainerName"] + "@" + table["SourceStorageAccountName"]  + ".dfs.core.windows.net"

            sourceManifestPath = table["SourceChangeFeedManifestPath"]
            sourceFilePath = table["SourceChangeFeedFilePath"]

            sourceManifestPath = abfssString + "/" + sourceManifestPath
            
            revsourceManifestPath = sourceManifestPath[len(sourceManifestPath)::-1]
            ordVal = revsourceManifestPath.find("/")
            substringrevsourceManifestPath = right(revsourceManifestPath,(len(revsourceManifestPath) -ordVal - 1))
            substringsourceManifestPath = substringrevsourceManifestPath[len(substringrevsourceManifestPath)::-1]
            df1 = spark.read.json(sourceManifestPath, multiLine=True)
            entityPaths = df1.select("entities").limit(1).collect()[0][0]
            
            for entity in entityPaths:        
                if entity["entityName"]==tableName:

                    entityName = entity["entityName"]
                    entityPath = entity["entityPath"]
                    entityManifestPath = left(entityPath,(len(entityPath) - (len(entityName)+1)))
                    entityConfigurationPath = substringsourceManifestPath + "/"+ entityManifestPath
                    jsonConfigs = spark.read.json(entityConfigurationPath, multiLine=True)
                    definitions = jsonConfigs.select("definitions").limit(1).collect()[0][0]           
                    
                    
                    for rows in definitions:
                        
                        numAttributes = len(rows['hasAttributes'])
                        
                        loopCount = 0
                        while loopCount < numAttributes:
                            try:
                                constantValues = json.loads(rows['hasAttributes'][loopCount]['appliedTraits'][3])
                                fieldName = rows['hasAttributes'][loopCount]['name']
                                #print((rows['hasAttributes'][5]['appliedTraits']))
                                loopCount = loopCount+1
                                rowCount = len(constantValues['arguments'][1]['value']['entityReference']['constantValues'])
                                i = 0
                                while i < rowCount:
                                    unumList = constantValues['arguments'][1]['value']['entityReference']['constantValues'][i]
                                    unumList.insert(0,tableName)
                                    unumList.insert(1,fieldName)
                                    unumList.append(ETLBatchSeriesID)
                                    unumList.append(DMLAction)
                                    enumValues.append(unumList)
                                    i=i+1
                            except:
                                loopCount = loopCount + 1

        schema = StructType([ \
            StructField("TableName",StringType(),True), \
            StructField("FieldName",StringType(),True), \
            StructField("Constant0",StringType(),True), \
            StructField("FormDisplayName",StringType(),True), \
            StructField("Value",StringType(),True), \
            StructField("Key",StringType(),True), \
            StructField("ETLBatchSeriesID",LongType(),True),
            StructField("DML_Action",StringType(),True) ])


        rdd = sc.parallelize(enumValues)
        dfFullBronzeConfigurations = sqlContext.createDataFrame(rdd, schema)  

        dfFullBronzeConfigurations.createOrReplaceTempView("vwConstantEntity")
        dfFullBronzeConfigurations = spark.sql("SELECT DISTINCT * FROM vwConstantEntity")
        sqlCommand = "MERGE INTO " + bronzedatabaseName + ".ConstantEntity T \
        USING vwConstantEntity S \
        ON T.TableName = S.TableName AND T.FieldName = S.FieldName AND S.Constant0 = T.Constant0 AND S.FormDisplayName = T.FormDisplayName AND S.Value = T.Value AND S.Key = T.Key \
        WHEN NOT MATCHED THEN INSERT *"
        spark.sql(sqlCommand)

In [ ]:
if isInitialSetup != 1:
    df=spark.sql("SELECT REPLACE(SourceQuery,'@REPLACEBATCH@',"+str(ETLBatchSeriesID)+") AS SourceQuery, TableName, MergeJoinKey, Database FROM %s.SilverMeta WHERE IsActive = 1 and lower(TableName) IN (%s,'constantentity') AND PipelineGroup = '%s'" % (configsdatabaseName,str(tableList).replace('[','').replace(']','').lower(),pipelineGroup))
    silverdfcollect = df.collect()   

In [ ]:
from concurrent.futures import ThreadPoolExecutor
from datetime import datetime
from pyspark.sql.types import StructType,StructField, StringType, IntegerType
import json
resultList = []
def main():
    notebooks = list(gather_notebookcalls(silverdfcollect,isFullLoad))
    with ThreadPoolExecutor(max_workers=parallelWorkers) as Thready:
        #print("Thready")
        Thready.map(call_notebooks, notebooks)    
        
        
"""def call_notebooks(notebook_par: str) -> None:
    notebookParams = json.loads(notebook_par)
    #print(notebookParams["parameters"])
    res = mssparkutils.notebook.run(notebookParams["path"], notebookParams["timeout"], notebookParams["parameters"])
    resultList.append(res)"""

def call_notebooks(notebook_par: str) -> None:
    retry = retryAttempts
    notebookParams = json.loads(notebook_par)
    #print(notebookParams["parameters"])
    res=""
    try:
        spark.conf.set("spark.synapse.nbs.kernelid", "")
        res = mssparkutils.notebook.run(notebookParams["path"], notebookParams["timeout"], notebookParams["parameters"])
        resultList.append(res)
        print(res)
        IsRetry = 0
    except Exception as e:
        print(e)
        IsRetry = 1
    if IsRetry == 1 and retry > 0:
            print("Retrying")
            spark.conf.set("spark.synapse.nbs.kernelid", "")
            res = mssparkutils.notebook.run(notebookParams["path"], notebookParams["timeout"], notebookParams["parameters"])
            resultList.append(res)
            print(res) 
            retry = retry - 1
    else:
        raise ValueError("ERROR DURING PROCESSING. CHECK YOUR LOGS. " + notebookParams["parameters"])

def gather_notebookcalls(silverdfcollect,isFullLoad) -> iter:  
    retryAttempts = 1
    notebooks = []
    #isFullLoad = 0
    schema = StructType([ \
        StructField("path",StringType(),True), \
        StructField("timeout",IntegerType(),True), \
        StructField("parameters",StructType([ \
            StructField("isFullLoad", IntegerType(), True),  \
            StructField("databaseName", StringType(), True), \
            StructField("tableName", StringType(), True), \
            StructField("ETLBatchSeriesID", StringType(), True), \
            StructField("mergeKey", StringType(), True), \
            StructField("SourceQuery", StringType(), True)])), \
        StructField("retry",StringType(),True)])

    for row in silverdfcollect:
        tableName = row['TableName']
        MergeJoinKey = str(row['MergeJoinKey'])
        sourceQuery = row['SourceQuery']
        notebooks.append(("/BronzeToSilver",5000,(isFullLoad,row['Database'],row['TableName'],ETLBatchSeriesID,MergeJoinKey,sourceQuery),retryAttempts)) 
    
    rdd = sc.parallelize(notebooks)
    dfSilverConfigurations = sqlContext.createDataFrame(rdd, schema)  
    notebookCollect = dfSilverConfigurations.toJSON().collect()
    for notebook in notebookCollect:
        yield notebook

if __name__ == '__main__':
    t1 = datetime.now()
    if isInitialSetup != 1:
        main()
    t2 = datetime.now()
    x = t2-t1
    print(f'It took {x} to process files')

In [ ]:
updateSilverMeta = "UPDATE %s.silvermeta \
SET ETLBatchSeriesID = %s \
WHERE IsActive = 1 and lower(TableName) IN (%s,'constantentity') AND PipelineGroup = '%s'" % (configsdatabaseName,str(ETLBatchSeriesID),str(tableList).replace('[','').replace(']','').lower(),pipelineGroup)

spark.sql(updateSilverMeta)

if isInitialSetup != 1:
    updateBatch = "UPDATE %s.etlbatchseries \
    SET IsProcessingComplete = 1 \
    WHERE PipelineGroup = '%s' \
    AND ETLBatchSeriesID = %i" % (configsdatabaseName,pipelineGroup,ETLBatchSeriesID)
    spark.sql(updateBatch)
